In [28]:
!pip install folium geopy timezonefinder pytz
!pip install opening_hours_py

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)


In [ ]:
import gpxpy
import folium
import requests
import json
import datetime
from geopy.distance import geodesic
# Auf eine moderne, aktiv gepflegte Bibliothek umgestellt
from opening_hours.opening_hours import OpeningHours
from timezonefinder import TimezoneFinder
# FIX: Von der veralteten 'pytz' zur modernen, eingebauten 'zoneinfo' Bibliothek gewechselt
from zoneinfo import ZoneInfo, ZoneInfoNotFoundError

# @title GPX-Routen-Visualisierer mit POI-Ankunftszeitanalyse
# @markdown ---
# @markdown **Abhängigkeiten:**
# @markdown Führen Sie `pip install gpxpy folium requests opening-hours-py timezonefinder geopy` in Ihrem Terminal aus. `pytz` wird nicht mehr benötigt.
# @markdown
# @markdown **Anleitung:**
# @markdown 1. Tragen Sie Ihre Planungsdaten (Geschwindigkeit, Startdatum und -zeit) ein.
# @markdown 2. Ersetzen Sie `'PFAD/ZU/IHRER/DATEI.gpx'` mit dem tatsächlichen Pfad zu Ihrer GPX-Datei.
# @markdown 3. Führen Sie die Zelle aus.
# @markdown ---

# @markdown ### Parameter für die Routenplanung
average_speed_kmh = 20.0 # @param {type:"number"}
# @markdown Geben Sie Ihr Startdatum und Ihre Startzeit in Ihrer **lokalen Zeitzone** (UTC+2) an.
start_date = "2025-07-05" # @param {type:"date"}
start_time_str = "10:00" # @param {type:"string"}


def find_closest_point_on_track(poi_coords, track_points_with_dist):
    """
    Findet den nächstgelegenen Punkt auf der Route zu einem gegebenen POI.
    `track_points_with_dist` ist eine Liste von Tupeln: (point, distance_from_start).
    """
    min_dist = float('inf')
    closest_point_info = None
    for point, distance_from_start in track_points_with_dist:
        dist = geodesic(poi_coords, (point.latitude, point.longitude)).meters
        if dist < min_dist:
            min_dist = dist
            # Speichert das Tupel, das sowohl den Punkt als auch die Distanz enthält
            closest_point_info = (point, distance_from_start)
    return closest_point_info

def fetch_and_add_pois(folium_map, gpx, average_speed_kmh, start_datetime_utc):
    """
    Fragt die Overpass API nach POIs entlang der Route ab, prüft die Öffnungszeiten
    und fügt sie zur Karte hinzu.
    """
    print("Suche nach Geschäften entlang der Route...")
    print(f"Verwende festgelegte Startzeit (UTC): {start_datetime_utc.strftime('%Y-%m-%d %H:%M:%S')}")

    # Korrekte Methode zur Berechnung der kumulativen Distanzen
    track_points_with_dist = []
    dist_so_far = 0.0
    for track in gpx.tracks:
        for segment in track.segments:
            for i, point in enumerate(segment.points):
                if i > 0:
                    dist_so_far += point.distance_2d(segment.points[i-1])
                track_points_with_dist.append((point, dist_so_far))
    
    all_track_points = [p[0] for p in track_points_with_dist]
    
    CHUNK_SIZE = 150
    processed_poi_ids = set()
    total_pois_found = 0
    poi_group = folium.FeatureGroup(name="Geschäfte", show=True)
    tf = TimezoneFinder()

    num_chunks = (len(all_track_points) + CHUNK_SIZE - 1) // CHUNK_SIZE
    print(f"Lange Route erkannt. Wird in {num_chunks} Abschnitte aufgeteilt.")

    for i in range(0, len(all_track_points), CHUNK_SIZE):
        chunk = all_track_points[i:i + CHUNK_SIZE + 1]
        print(f"Verarbeite Abschnitt {i // CHUNK_SIZE + 1} von {num_chunks}...")

        flat_coords = [str(coord) for point in chunk for coord in (point.latitude, point.longitude)]
        points_str = ",".join(flat_coords)
        
        overpass_query = f"""
        [out:json][timeout:90];
        (
          nwr(around:1000,{points_str})["amenity"="fuel"];
          nwr(around:1000,{points_str})["shop"="supermarket"];
          nwr(around:1000,{points_str})["shop"="convenience"];
        );
        out center;
        """
        
        overpass_url = "https://overpass-api.de/api/interpreter"
        try:
            response = requests.post(overpass_url, data={'data': overpass_query})
            response.raise_for_status()
            data = response.json()
            
            if not data['elements']: continue

            for element in data['elements']:
                element_id = element['id']
                if element_id in processed_poi_ids: continue
                
                processed_poi_ids.add(element_id)
                total_pois_found += 1
                
                lat = element.get('lat') or element.get('center', {}).get('lat')
                lon = element.get('lon') or element.get('center', {}).get('lon')
                if not lat or not lon: continue
                
                closest_point_tuple = find_closest_point_on_track((lat, lon), track_points_with_dist)
                if not closest_point_tuple: continue
                
                distance_to_poi_m = closest_point_tuple[1]
                
                speed_avg_ms = (average_speed_kmh * 1000) / 3600
                speed_min_ms = speed_avg_ms * 0.9
                speed_max_ms = speed_avg_ms * 1.1

                travel_time_avg_s = distance_to_poi_m / speed_avg_ms if speed_avg_ms > 0 else 0
                travel_time_earliest_s = distance_to_poi_m / speed_max_ms if speed_max_ms > 0 else 0
                travel_time_latest_s = distance_to_poi_m / speed_min_ms if speed_min_ms > 0 else 0
                
                eta_avg_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_avg_s)
                eta_earliest_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_earliest_s)
                eta_latest_utc = start_datetime_utc + datetime.timedelta(seconds=travel_time_latest_s)

                opening_hours_str = element.get('tags', {}).get('opening_hours')
                status_text = "<i>Öffnungszeiten unbekannt</i>"
                
                eta_avg_local = eta_avg_utc # Fallback
                eta_earliest_local = eta_earliest_utc
                eta_latest_local = eta_latest_utc

                if opening_hours_str:
                    try:
                        tz_str = tf.timezone_at(lng=lon, lat=lat)
                        if tz_str:
                           # FIX: Verwendet die moderne zoneinfo-Bibliothek
                           local_tz = ZoneInfo(tz_str)
                           eta_avg_local = eta_avg_utc.astimezone(local_tz)
                           eta_earliest_local = eta_earliest_utc.astimezone(local_tz)
                           eta_latest_local = eta_latest_utc.astimezone(local_tz)

                           oh = OpeningHours(opening_hours_str)
                           # Die Bibliothek erwartet ein timezone-aware datetime-Objekt
                           is_open = oh.is_open(eta_avg_local)
                           
                           status_color = 'green' if is_open else 'red'
                           status_word = 'Geöffnet' if is_open else 'Geschlossen'
                           status_text = f"<strong style='color:{status_color};'>{status_word}</strong> bei Ankunft"
                        else:
                           status_text = "<i>Zeitzone unbekannt</i>"
                    except ZoneInfoNotFoundError:
                        status_text = "<i>Fehler: Zeitzone konnte nicht gefunden werden.</i>"
                    except Exception as e:
                        status_text = f"<i>Fehler: {e}</i>"

                poi_type = "Unbekannt"
                poi_name = element.get('tags', {}).get('name', 'Unbenannt')
                icon_color = 'gray'; icon_symbol = 'question'
                tags = element.get('tags', {})
                if tags.get('amenity') == 'fuel': poi_type, icon_color, icon_symbol = 'Tankstelle', 'red', 'tint'
                elif tags.get('shop') == 'supermarket': poi_type, icon_color, icon_symbol = 'Supermarkt', 'blue', 'shopping-cart'
                elif tags.get('shop') == 'convenience': poi_type, icon_color, icon_symbol = 'Kiosk', 'green', 'shopping-basket'

                popup_html = f"""
                <b>{poi_type}: {poi_name}</b><br>
                <hr style='margin: 3px 0;'>
                <b>Status:</b> {status_text}<br>
                <b>Ankunft (ca.):</b> {eta_avg_local.strftime('%A, %H:%M')} Uhr<br>
                <small><i>Fenster: {eta_earliest_local.strftime('%H:%M')} - {eta_latest_local.strftime('%H:%M')}</i></small><br>
                <small>Regel: {opening_hours_str or 'n/a'}</small>
                """
                folium.Marker(location=[lat, lon], popup=popup_html, icon=folium.Icon(color=icon_color, icon=icon_symbol, prefix='fa')).add_to(poi_group)
        
        except requests.exceptions.RequestException as e:
            print(f"Fehler bei Abschnitt {i // CHUNK_SIZE + 1}: {e}")
        except json.JSONDecodeError:
            print(f"Fehler bei Abschnitt {i // CHUNK_SIZE + 1}: Ungültige Antwort vom Server.")
            
    if total_pois_found > 0:
        print(f"\nInsgesamt {total_pois_found} einzigartige Geschäfte gefunden und zur Karte hinzugefügt.")
        poi_group.add_to(folium_map)
    else:
        print("\nKeine Geschäfte entlang der gesamten Route gefunden.")

def render_gpx_on_map(gpx_file_path, average_speed_kmh, start_date_str, start_time_str):
    """Liest eine GPX-Datei, rendert die Route und fügt POIs hinzu."""
    try:
        # Verwendet die lokale Zeitzone des Systems, um die Benutzereingabe zu interpretieren
        naive_datetime = datetime.datetime.strptime(f"{start_date_str} {start_time_str}", "%Y-%m-%d %H:%M")
        # Wandelt die lokale Startzeit in UTC um, da alle Berechnungen in UTC erfolgen
        start_datetime_local = naive_datetime.astimezone()
        start_datetime_utc = start_datetime_local.astimezone(datetime.timezone.utc)


        with open(gpx_file_path, 'r', encoding='utf-8') as gpx_file:
            gpx = gpxpy.parse(gpx_file)

        points = []
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append(tuple([point.latitude, point.longitude]))

        if not points: print("Fehler: Keine Punkte in der GPX-Datei gefunden."); return

        map_center = points[0]
        m = folium.Map(location=map_center, zoom_start=13, tiles='OpenStreetMap')
        folium.PolyLine(points, color="#3388ff", weight=5, opacity=0.8, popup=gpx.tracks[0].name).add_to(m)

        fetch_and_add_pois(m, gpx, average_speed_kmh, start_datetime_utc)
        
        folium.LayerControl().add_to(m)
        sw = min(points, key=lambda x: x[0])[0], min(points, key=lambda x: x[1])[1]
        ne = max(points, key=lambda x: x[0])[0], max(points, key=lambda x: x[1])[1]
        m.fit_bounds([sw, ne])

        print(f"\nErfolg! Route '{gpx.tracks[0].name or 'Unbenannte Route'}' wird dargestellt.")
        return m

    except FileNotFoundError:
        print(f"Fehler: Die Datei wurde nicht unter '{gpx_file_path}' gefunden.")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# --- Hier den Pfad zu Ihrer GPX-Datei eintragen ---
gpx_file_path = 'data/2025-06-09_2312053799_Maurice Brocco Vanilla(1).gpx'

# Führt die Funktion mit den neuen Parametern aus und zeigt die Karte an
gpx_map = render_gpx_on_map(gpx_file_path, average_speed_kmh, start_date, start_time_str)
gpx_map


Suche nach Geschäften entlang der Route...
Verwende festgelegte Startzeit (UTC): 2025-07-05 08:00:00
Lange Route erkannt. Wird in 61 Abschnitte aufgeteilt.
Verarbeite Abschnitt 1 von 61...
Verarbeite Abschnitt 2 von 61...
Verarbeite Abschnitt 3 von 61...
Verarbeite Abschnitt 4 von 61...
Verarbeite Abschnitt 5 von 61...
Verarbeite Abschnitt 6 von 61...
Verarbeite Abschnitt 7 von 61...
Verarbeite Abschnitt 8 von 61...
Verarbeite Abschnitt 9 von 61...
Verarbeite Abschnitt 10 von 61...
Verarbeite Abschnitt 11 von 61...
Verarbeite Abschnitt 12 von 61...
Verarbeite Abschnitt 13 von 61...
Verarbeite Abschnitt 14 von 61...
Verarbeite Abschnitt 15 von 61...
Verarbeite Abschnitt 16 von 61...
Verarbeite Abschnitt 17 von 61...
